In [1]:
import sys
import pandas as pd

In [10]:
# Get the paths
path_A = '../DATA/imdb3_neg_nan.csv'
path_B = '../DATA/thenumbers3_neg_nan.csv'
path_C = '../DATA/AllTuplePairs.csv'
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')
C = em.read_csv_metadata(path_C, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')


In [11]:
C.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
0,4146,1757,11,Hamlet,1996,PG-13,242 min,Drama,Kenneth Branagh,"Kenneth Branagh,Julie Christie,Derek Jacobi,Kate Winslet",$4.41M,Hamlet,1996,PG-13,-1,Drama,-1,Kenneth Branagh,"$7,129,670"
1,6671,1093,20,The Assassination of Jesse James by the Coward Robert Ford,2007,R,160 min,"Biography, Crime, Drama",Andrew Dominik,"Brad Pitt,Casey Affleck,Sam Shepard,Mary-Louise Parker",$3.90M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1
2,6891,1723,20,Sweeney Todd: The Demon Barber of Fleet Street,2007,R,116 min,"Drama, Horror, Musical",Tim Burton,"Johnny Depp,Helena Bonham Carter,Alan Rickman,Timothy Spall",$52.90M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1
3,7619,3795,20,30 Days of Night,2007,R,113 min,"Horror, Thriller",David Slade,"Josh Hartnett,Melissa George,Danny Huston,Ben Foster",$39.57M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1
4,7647,3883,20,In the Valley of Elah,2007,R,121 min,"Crime, Drama, Mystery",Paul Haggis,"Tommy Lee Jones,Charlize Theron,Jonathan Tucker,Jason Patric",$6.78M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1


In [15]:
G = em.read_csv_metadata('../DATA/G.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
#Build and train model on G
rf = em.RFMatcher(name='RF', random_state=0)
feature_table = em.get_features_for_matching(A, B,
                             validate_inferred_attr_types=False)
H = em.extract_feature_vecs(G, 
                            feature_table=feature_table, 
                            attrs_after='gold',
                            show_progress=False)
rf.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')



In [17]:
#convert All Tuple Pairs to features
CF = em.extract_feature_vecs(C, 
                            feature_table=feature_table, 
                            #attrs_after='gold',
                            show_progress=False)


In [29]:
#Save matching and non matching pairs
#apply to C4
predictions = rf.predict(table=CF,#The entire evaluation set
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id'], 
              append=True, target_attr='predicted', inplace=False)

In [32]:
P = predictions.predicted
CP = C[P==1]
#tit = [C.ltable_title, C.rtable_title]
allData = pd.concat([CP.ltable_id, CP.ltable_title, CP.rtable_id, CP.rtable_title],axis =1)
allData.to_csv('../DATA/MatchPredctionsOnAllTuplePairs.csv', index=False)

In [33]:
allData

,ltable_id,ltable_title,rtable_id,rtable_title
0,1757,Hamlet,11,Hamlet
6,3547,Home for the Holidays,22,Home for the Holidays
14,502,Hot Fuzz,36,Hot Fuzz
17,3688,Drop Dead Gorgeous,57,Drop Dead Gorgeous
30,2558,The Butterfly Effect,123,The Butterfly Effect
35,4100,Mystic Pizza,135,Mystic Pizza
36,1135,Mystic River,139,Mystic River
37,114,Butch Cassidy and the Sundance Kid,141,Butch Cassidy and the Sundance Kid
39,2498,Mystery Science Theater 3000: The Movie,154,Mystery Science Theatre 3000
40,4076,Mystery Team,156,Mystery Team
